In [71]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [42]:
%sendtofile --source UDPblackviewphone.py


Sent 30 lines (819 bytes) to UDPblackviewphone.py.


In [72]:
%sendtofile --source UDPblackviewphone.py
%sendtofile --source ubxminimal.py

Sent 66 lines (2004 bytes) to ubxminimal.py.


In [42]:
%sendtofile deviceletter.txt
C

Sent 1 lines (1 bytes) to deviceletter.txt.


In [52]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [63]:
%sendtofile main.py

import time, machine

# delay long enough for Ctrl-C before the wlan bricks it for debugging
pled = machine.Pin(2, machine.Pin.OUT)
for i in range(11):
    pled.value(i%2)
    time.sleep_ms(500)  

from ubxminimal import uartUBX, initUBX
initUBX()

for i in range(11):
    pled.value(i%2)
    time.sleep_ms(200)

deviceletter = open("deviceletter.txt", "r").read()
print("deviceletter", deviceletter)

from UDPblackviewphone import connectActivePhone
while not connectActivePhone(pled):
    for j in range(3):
        for i in range(7):
            pled.value(i%2)
            time.sleep_ms(100)
        time.sleep_ms(800)

for i in range(21):
    pled.value(i%2)
    time.sleep_ms(80)

import socket
androidipnumber = "192.168.43.1"
portnumber = 9042

ubxbuffer = bytearray(1000)
mubxbuffer = memoryview(ubxbuffer)
while True:
    try:
        ss = socket.socket()
        ss.settimeout(1)
        print(ss)
        ss.connect(socket.getaddrinfo(androidipnumber, portnumber)[0][-1])
        s = ss.makefile('rwb', 0)
        print(s.readline())
        s.write(b"%c%c%c%c"%(deviceletter,deviceletter,deviceletter,deviceletter))
        while True:
            n = uartUBX.readinto(ubxbuffer)
            if n is not None:
                s.write(mubxbuffer[:n])
                pled.value(1-pled.value())
            else:
                time.sleep_ms(5)
    except OSError as e:
        print("OSError", e)



Sent 55 lines (1411 bytes) to main.py.


In [88]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.10 on 2019-01-25; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f5def4951d0, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [98]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [2]:
deviceletter = open("deviceletter.txt", "r").read()
print(b"%c%c%c%c"%(deviceletter,deviceletter,deviceletter,deviceletter))

b'CCCC'


In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [3]:
# for testing UBX is wired in
import machine
uartUBX = machine.UART(1, baudrate=9600, rx=16, tx=17)
print(uartUBX.read())

None


In [4]:
for l in uartUBX:
    print(l)

b'$GNRMC,,V,,,,,,,,,,N*4D\r\n'
b'$GNVTG,,,,,,,,,N*2E\r\n'
b'$GNGGA,,,,,,0,00,99.99,,,,,,*56\r\n'
b'$GNGSA,A,1,,,,,,,,,,,,,99.99,99.99,99.99*2E\r\n'
b'$GNGSA,A,1,,,,,,,,,,,,,99.99,99.99,99.99*2E\r\n'
b'$GPGSV,1,1,00*79\r\n'
b'$GLGSV,1,1,00*65\r\n'
b'$GNGLL,,,,,,V,N*7A\r\n'
b'$GNZDA,,,,,00,00*56\r\n'
b'$GNRMC,,V,,,,,,,,,,N*4D\r\n'
b'$GNVTG,,,,,,,,,N*2E\r\n'
b'$GNGGA,,,,,,0,00,99.99,,,,,,*56\r\n'
b'$GNGSA,A,1,,,,,,,,,,,,,99.99,99.99,99.99*,1,,,,,,,,,,,,,99.99,99.99,99.99*2E\r\n'
b'$GPGSV,1,1,00*79\r\n'
b'$GLGSV,1,1,00*65\r\n'
b'$GNGLL,,,,,,V,N*7A\r\n'
b'$GNZDA,,,,,00,00*56\r\n'


In [ ]:
%sendtofile main.py

import time, machine, urandom

# delay long enough to interrupt if we need to, 
# before the uart or wlan bricks it for debugging
pled = machine.Pin(2, machine.Pin.OUT)
for i in range(11):
    pled.value(i%2)
    time.sleep_ms(500)  

from ubxminimal import uartUBX, initUBX
initUBX()

for i in range(11):
    pled.value(i%2)
    time.sleep_ms(200)

deviceletter = open("deviceletter.txt", "r").read()
port = {"A":9020, "B":9021, "C":9022}[deviceletter]
udpaddr = ("192.168.43.1", port)  # ip default for android

from UDPblackviewphone import connectBlackview, connectS5
if deviceletter == 'C':
    connectS5(pled)
else:
    connectBlackview(pled)

for i in range(11):
    pled.value(i%2)
    time.sleep_ms(100)


import socket
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.settimeout(0.005)
    
ubxbuffer = bytearray(1000)

# main loop
PCudpaddr = None
reccount = 0
while True:
    n = uartUBX.readinto(ubxbuffer)
    if n is not None:
        try:
            s.sendto(mubxbuffer[:n+2], udpaddr)
            if PCudpaddr is not None:
                s.sendto(mubxbuffer[:n+2], PCudpaddr)
        except OSError as e:
            print("dwrite", e)
        #print(x[:5])
        pled.value(1-pled.value())
    else:
        try:
            mess, PCudpaddr = s.recvfrom(50)
            print(mess, PCudpaddr)
        except OSError as e:
            pass # timeout

